<a href="https://colab.research.google.com/github/biwnunticha/MT-for-songs-EN-TH/blob/main/MT5_Translation_from_songs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install simpletransformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 250 kB 23.7 MB/s 
     |████████████████████████████████| 1.9 MB 87.7 MB/s 
     |████████████████████████████████| 43 kB 2.4 MB/s 
     |████████████████████████████████| 7.6 MB 81.8 MB/s 
     |████████████████████████████████| 9.2 MB 93.3 MB/s 
     |████████████████████████████████| 451 kB 96.2 MB/s 
     |████████████████████████████████| 5.8 MB 98.6 MB/s 
     |████████████████████████████████| 1.3 MB 98.3 MB/s 
     |████████████████████████████████| 182 kB 100.2 MB/s 
     |████████████████████████████████| 168 kB 105.5 MB/s 
     |████████████████████████████████| 182 kB 93.2 MB/s 
     |████████████████████████████████| 62 kB 1.4 MB/s 
     |████████████████████████████████| 168 kB 90.0 MB/s 
     |████████████████████████████████| 166 kB 101.9 MB/s 
     |████████████████████████████████| 166 kB 102.6 MB/s 
     |█████████████████████████

In [ ]:
import logging
import pandas as pd
import json
from simpletransformers.t5 import T5Model, T5Args


logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [ ]:
!gdown --id 11bFsm_M-VXpKBXgoNanDq7_3cfk8yC_0

/usr/local/lib/python3.8/dist-packages/gdown/cli.py:127: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=11bFsm_M-VXpKBXgoNanDq7_3cfk8yC_0
To: /content/lyrics_attacut.csv
100% 41.8M/41.8M [00:00<00:00, 153MB/s]


In [ ]:
data = pd.read_csv('/content/lyrics_attacut.csv', index_col=False)

In [ ]:
data = data.dropna(subset=['lyrics', 'tokenized'])

In [ ]:
data.columns=['prefix', 'input_text','target_text']

In [ ]:
data['prefix'] ='translate english to thai'

In [ ]:
data

,prefix,input_text,target_text
0,translate english to thai,I washed my hands of us at the club,ฉัน ล้าง มือ ที่ เรา จับ มือ กัน ที่ ผับ
1,translate english to thai,You made a mess of me,เธอ ทำ ให้ ฉัน เละเทะ มาก เลย นะ
2,translate english to thai,I pictured you with other girls in love,ฉัน จินตนาการภาพ เธอ พลอดรัก กับ ผู้ หญิง คน อื่น
3,translate english to thai,Then threw up on the street,แล้ว ฉัน ก็ อ้วก ออก มา บน ถนน เลย
4,translate english to thai,Like waiting for a bus that never shows,มัน เหมือน การ รอ รถ เมล์ ที่ ไม่ มี วัน มา
...,...,...,...
287846,translate english to thai,I wish he never looked at me that way,อย่า มอง ฉัน อย่าง นั้น สิ
287847,translate english to thai,Eh eh eh eh,eh eh eh eh
287848,translate english to thai,There is nothing else I can say,ไม่ รู้ จะ พูด อะไร แล้ว อะะะะ
287849,translate english to thai,Eh eh eh eh,eh eh eh eh


In [ ]:
from sklearn.model_selection import train_test_split
train, rest = train_test_split(data, test_size=0.01)
validation, test = train_test_split(rest, test_size=0.5)

In [ ]:
train = train.reset_index().drop(columns='index')
validation = validation.reset_index().drop(columns='index')
test = test.reset_index().drop(columns='index')

In [ ]:
train_df = train.astype(str)
eval_df = validation.astype(str)

In [ ]:
train_df["prefix"] = ""
eval_df["prefix"] = ""

In [ ]:
!nvidia-smi

Mon Dec  5 12:01:10 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   66C    P0    28W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import torch
torch.cuda.is_available()

True

In [ ]:
model_args = T5Args()
model_args.max_seq_length = 96
model_args.train_batch_size = 8
model_args.eval_batch_size = 8
model_args.num_train_epochs = 1
model_args.evaluate_during_training = True
model_args.evaluate_during_training_steps = 30000
model_args.use_multiprocessing = False
model_args.fp16 = False
model_args.save_steps = -1
model_args.save_eval_checkpoints = False
model_args.no_cache = True
model_args.reprocess_input_data = True
model_args.overwrite_output_dir = True
model_args.preprocess_inputs = False
model_args.num_return_sequences = 1
#model_args.wandb_project = "MT5 EN-TH Translation"

model = T5Model("mt5", "google/mt5-base", args=model_args, use_cuda=True)

Downloading:   0%|          | 0.00/702 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.33G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/376 [00:00<?, ?B/s]

In [ ]:
model.train_model(train_df, eval_data=eval_df)

  0%|          | 0/284952 [00:00<?, ?it/s]

/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:3704: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and targets.

Here is a short example:

model_inputs = tokenizer(src_texts, text_target=tgt_texts, ...)

If you either need to use different keyword arguments for the source and target texts, you should do two calls like
this:

model_inputs = tokenizer(src_texts, ...)
labels = tokenizer(text_target=tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)
/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:3578: UserWarning: `as_target_tokenizer` is deprec

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Running Epoch 0 of 2:   0%|          | 0/35619 [00:00<?, ?it/s]

In [ ]:
model.predict(["I think I'm in love with you"])

In [ ]:
validation

In [ ]:
train_set = set(train['input_text'].to_list())

In [ ]:
test_list = test['input_text'].tolist()
gold_list = test['target_text'].tolist()

In [ ]:
#คัดจาก train ออก
for index, test in enumerate(test_list):
  if test in train_set:
    test_list.pop(index)
    gold_list.pop(index)

In [ ]:
new_test = pd.DataFrame({'lyrics': test_list, 'tokenized': gold_list})

In [ ]:
train.to_csv('train.csv')
new_test.to_csv('test.csv')
validation.to_csv('dev.csv')

In [ ]:
!pip install evaluate

In [ ]:
!pip install sacrebleu

In [ ]:
import evaluate

In [ ]:
# Predict
pred_list = model.predict(test_list)

In [ ]:
chrf = evaluate.load("chrf")
results = chrf.compute(predictions=pred_list,references=gold_list,word_order=2,lowercase=True)
print(results)

In [ ]:
pred_df = pd.DataFrame({'input_text': test_list, 'predict': [x.replace(' ','') for x in pred_list], 'gold': [x.replace(' ','') for x in gold_list]})

In [ ]:
pred_df.to_csv('mt5_trans.csv')

In [ ]:
pred_df

In [ ]:
train.to_csv('train.csv')

In [ ]:
print (len(new_test))

In [ ]:
test_sort.to_csv('test_this.csv')